# Training Model using L_CNN Architecture

This notebook covers the first phase of training the models using various Hydropathy scales
The code in `generate_hr.ipynb` and `code/generate_hr.py` are used to generate multiple Hr files based on varius hydropathy scale

Following that In this notebook, we use all the generated Hr files to train the model and check accuracy as well as save the results.

We are using the already given L_CNN architecture without tuning to train models. There are 28 Hr scales thus we will have 28 model trained and their accuracy will be saved in a CSV.

## Create the perfect structure

First let's create the training and testing data strcuture


In [1]:
#os.makedirs("../generated_data/train/", exist_ok=True)

#for file_path in tqdm(os.listdir(directory)):


In [2]:
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [3]:
os.makedirs("../generated_data/training_folders/", exist_ok=True)

In [4]:
# read the data
# read text file as numpy array

# training files
filename = '../dataset/train/dataset.txt'
dataset = pd.read_csv(filename)

filename = "../dataset/train/shape.txt"
sh = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/train/dist.txt"
dist = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/train/el.txt"
el = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/train/classification.txt"
classification = pd.read_csv(filename,delimiter = " ", header=None)


In [5]:
# read the data
# read text file as numpy array

# test files
filename = '../dataset/test/dataset.txt'
dataset_test = pd.read_csv(filename)

filename = "../dataset/test/shape.txt"
sh_test = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/test/dist.txt"
dist_test = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/test/el.txt"
el_test = pd.read_csv(filename,delimiter = " ", header=None)

filename = "../dataset/test/classification.txt"
classification_test = pd.read_csv(filename,delimiter = " ", header=None)


In [6]:
# for training set
directory = "../generated_data/train/Hr"
for file_path in tqdm(os.listdir(directory)):
    folder_name = file_path.split(".")[0]
    #print(folder_name)
    os.makedirs(f"../generated_data/training_folders/{folder_name}", exist_ok=True)
    os.makedirs(f"../generated_data/training_folders/{folder_name}/train", exist_ok=True)
    os.makedirs(f"../generated_data/training_folders/{folder_name}/test", exist_ok=True)

    # read hr file
    hr = pd.read_csv(directory + "/" + file_path,delimiter = " ", header=None)

    # read

    # for training set--------------------------------
    # save dataset.txt in all the train folders
    dataset.to_csv(f'../generated_data/training_folders/{folder_name}/train/dataset.txt', sep=',', index=False)

    # save shape.txt in all the train folders
    sh.to_csv(f'../generated_data/training_folders/{folder_name}/train/shape.txt', sep=' ', header=None, index=False)

    # save dist.txt in all the train folders
    dist.to_csv(f'../generated_data/training_folders/{folder_name}/train/dist.txt', sep=' ',header=None, index=False)
    
    # save el.txt in all the train folders
    el.to_csv(f'../generated_data/training_folders/{folder_name}/train/el.txt', sep=' ',header=None, index=False)

    # save hr.txt in all the train folders
    hr.to_csv(f'../generated_data/training_folders/{folder_name}/train/hr.txt', sep=' ',header=None, index=False)

    # save classification.txt in all the train folders
    classification.to_csv(f'../generated_data/training_folders/{folder_name}/train/classification.txt', sep=' ',header=None, index=False)


# for training set
directory_test = "../generated_data/test/Hr_modified_gaussian_Power_transformed"
for file_path in tqdm(os.listdir(directory_test)):
    folder_name = file_path.split(".")[0]
    os.makedirs(f"../generated_data/training_folders/{folder_name}/test", exist_ok=True)

    # read hr file
    hr_test = pd.read_csv(directory_test + "/" + file_path,delimiter = " ", header=None)

    # for test set--------------------------------
    # save dataset.txt in all the train folders
    dataset_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/dataset.txt', sep=',', index=False)

    # save shape.txt in all the train folders
    sh_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/shape.txt', sep=' ',header=None, index=False)

    # save dist.txt in all the train folders
    dist_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/dist.txt', sep=' ',header=None, index=False)
    
    # save el.txt in all the train folders
    el_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/el.txt', sep=' ',header=None, index=False)

    # save hr.txt in all the train folders
    hr_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/hr.txt', sep=' ',header=None, index=False)

    # save classification.txt in all the train folders
    classification_test.to_csv(f'../generated_data/training_folders/{folder_name}/test/classification.txt',header=None, sep=' ', index=False)

print("Folder Creation Done")

    

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

Folder Creation Done


### Training on various Hr files

#### Archive code

import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import MaxPooling1D
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import string
import re
import random
import hashlib
from tqdm.notebook import tqdm

def read_and_norm(dataset_path, type_,type):
    with open('{}/{}/{}.txt'.format(dataset_path, type_,type), 'rb') as f:
        matrix = [[float(x) for x in line.split()] for line in f]
    matrix = np.array(matrix)
    min_m = matrix.min().min()
    max_m = matrix.max().max()
    matrix = ((matrix - min_m) / (max_m - min_m))
    return matrix

def load_full_dataset(type_,dataset_path):
	classification = np.loadtxt('{}/{}/classification.txt'.format(dataset_path, type_))
	classification = np.array(classification).reshape(-1,1)

	with open('{}/{}/hr.txt'.format(dataset_path, type_), "r") as file:
		hr = []
		righe_con_9_colonne = []
		for indice, riga in enumerate(file):
			colonne = riga.split()
			if len(colonne) == 9:
				righe_con_9_colonne.append(indice)
			else:
				hr.append(colonne)
	hr = [[float(string) for string in inner] for inner in hr]
	hr = np.array(hr)


	shape = read_and_norm(dataset_path, type_,'shape')
	el = read_and_norm(dataset_path, type_,'el')
	dist = read_and_norm(dataset_path, type_,'dist')

	classification = np.delete(classification, righe_con_9_colonne, 0)
	shape = np.delete(shape, righe_con_9_colonne, 0)
	el = np.delete(el, righe_con_9_colonne, 0)
	dist = np.delete(dist, righe_con_9_colonne, 0)


	data_X = np.array([p for p in zip(shape, dist, el, hr)])
	data_X = data_X.reshape(data_X.shape[0], data_X.shape[1], data_X.shape[2], 1)

	return(data_X,classification)


# fit and evaluate a model
def evaluate_model_2dconv(trainX, trainy, testX, testy, save_model, model_path, file_path):
	verbose, epochs, batch_size = 0, 300, 1
	n_outputs = trainy.shape[1]
	model = Sequential()

	model.add(Conv2D(filters=9, kernel_size=(4,1), input_shape=trainX.shape[1:],activation='relu'))
	model.add(Dropout(0.25))
	model.add(Conv2D(filters=3, kernel_size=(1,3), activation='relu')) #(1,5)

	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(30, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(30, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(n_outputs, activation='sigmoid'))
	#model.summary()
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

	es = EarlyStopping(monitor='val_accuracy',
				   mode='max',
				   patience=50,
				   restore_best_weights=True)

	history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,validation_split=.2,callbacks=[es])#,callbacks=rlronp)#) #,callbacks=[es])

	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

	pred_label_ = model.predict(testX,batch_size=batch_size, verbose=0)

	pred_label = [1. if x >= 0.5 else 0. for x in pred_label_]
	results = pd.DataFrame({'Pred': pred_label,'Prob':pred_label_.reshape(-1), 'True': testy.reshape(-1)})



	if save_model:
		model.save('{}/{}.keras'.format(model_path,file_path))

	return accuracy, history


# summarize scores
def summarize_results(scores):
	#print(scores)
	m, s = mean(scores), std(scores)
	#print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
	return m

results = {"File":[],
		   "Scores":[],
		   "Best_Accuracy":[]}

# run an experiment
def run_experiment(repeats=1):
	save_model = 1
	model_path = '../models/1st_phase'

	directory = "../generated_data/training_folders"
	for file_path in tqdm(os.listdir(directory)):
		dataset_path = directory + "/" +file_path
			
		#load data
		trainX, trainy = load_full_dataset('train',dataset_path)
		testX, testy = load_full_dataset('test',dataset_path)
		#print(trainX.shape,trainy.shape,testX.shape,testy.shape)
		
		# repeat experiment
		scores = list()
		for r in range(repeats):
			score,history = evaluate_model_2dconv(trainX, trainy, testX, testy, save_model, model_path, file_path)

			score = score * 100.0
			#print('>#%d: %.3f' % (r+1, score))
			scores.append(score)

		#print(score)
		best_score = summarize_results(scores)
		
		# save it to dataframe
		results["File"].append(file_path)
		results["Scores"].append(scores)
		results["Best_Accuracy"].append(best_score)
		df = pd.DataFrame(results)
		max_accuracy_row = df.loc[df['Best_Accuracy'].idxmax()]
		print(max_accuracy_row)
		df.to_csv("../static/phase_1_results.csv", header=False)

run_experiment()

df = pd.DataFrame(results)
print(df.shape)
df.head()


# run an experiment
def run_experiment(repeats=1):
	save_model = 1
	model_path = '../models/1st_phase'

	directory = "../generated_data/training_folders"
	for file_path in tqdm(os.listdir(directory)):
		dataset_path = directory + "/" +file_path
			
		#load data
		trainX, trainy = load_full_dataset('train',dataset_path)
		testX, testy = load_full_dataset('test',dataset_path)
		#print(trainX.shape,trainy.shape,testX.shape,testy.shape)
		
		# repeat experiment
		scores = list()
		for r in range(repeats):
			score,history = evaluate_model_2dconv(trainX, trainy, testX, testy, save_model, model_path, file_path)

			score = score * 100.0
			#print('>#%d: %.3f' % (r+1, score))
			scores.append(score)

		#print(score)
		best_score = summarize_results(scores)
		
		# save it to dataframe
		results["File"].append(file_path)
		results["Scores"].append(scores)
		results["Best_Accuracy"].append(best_score)
		df = pd.DataFrame(results)
		max_accuracy_row = df.loc[df['Best_Accuracy'].idxmax()]
		print(max_accuracy_row)
		df.to_csv("../static/phase_1_results.csv", header=False)

run_experiment()

df = pd.DataFrame(results)
print(df.shape)
df.head()

There are 28 training phases..

what we can do is.. we train the model_1 with 1st Hr file..

Then use that model architecture as transfer learning to train the 2nd..

Following this patter we can train the data over multiple data sources to extract best of all the Hrs

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from tqdm.notebook import tqdm

In [8]:
# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [9]:
def read_and_minmax_file(dataset_path, type_, data_type, apply_noise=False, scaler=None):
    """
    Reads a .txt file with numerical data line-by-line.
    
    For training (scaler is None):
        - Optionally adds Gaussian noise (σ=0.4).
        - Computes the min and max values of the data and then applies Min–Max scaling.
        - Returns the scaled matrix and a dictionary with:
              'minmax': (min_val, max_val)
              
    For test data (scaler provided):
        - Does not add noise.
        - Uses the learned min–max values to perform scaling.
    
    Returns:
        scaled_matrix: The scaled data.
        scaler_params: Dictionary with min–max parameters.
        valid_indices: List of line indices that were successfully parsed.
    """
    file_path = os.path.join(dataset_path, type_, f"{data_type}.txt")
    matrix = []
    valid_indices = []
    with open(file_path, 'r') as f:
        for idx, line in enumerate(f):
            try:
                values = [float(x) for x in line.strip().split()]
                matrix.append(values)
                valid_indices.append(idx)
            except Exception as e:
                print(f"Skipping invalid line {idx+1} in {file_path}: {e}")
    matrix = np.array(matrix)
    
    # Add Gaussian noise only for training (increase noise to 0.4)
    if apply_noise:
        matrix = matrix + np.random.normal(loc=0.0, scale=0.16, size=matrix.shape)
    
    if scaler is None:
        # Compute min and max values for min-max scaling
        min_val = matrix.min()
        max_val = matrix.max()
        if max_val - min_val == 0:
            scaled = matrix
        else:
            scaled = (matrix - min_val) / (max_val - min_val)
        scaler_params = {'minmax': (min_val, max_val)}
    else:
        min_val, max_val = scaler['minmax']
        if max_val - min_val == 0:
            scaled = matrix
        else:
            scaled = (matrix - min_val) / (max_val - min_val)
        scaler_params = scaler  # unchanged for test
    
    return scaled, scaler_params, valid_indices


In [10]:
def read_and_minmax_hr(dataset_path, type_, apply_noise=False, scaler=None):
    """
    Special processing for hr.txt:
      - Skip lines that have exactly 9 columns.
      - For valid lines, parse numerical data.
    
    For training (scaler is None):
        - Optionally adds Gaussian noise.
        - Computes the min and max values of the data and then applies Min–Max scaling.
        - Returns the scaled data and a dictionary with:
              'minmax': (min_val, max_val)
    
    For test data (scaler provided):
        - Does not add noise.
        - Uses the learned min–max values to perform scaling.
    
    Returns:
        hr_scaled: The scaled HR data.
        scaler_params: Dictionary with scaling parameters.
        valid_indices: List of valid line indices.
    """
    file_path = os.path.join(dataset_path, type_, "hr.txt")
    hr_list = []
    valid_indices = []
    with open(file_path, 'r') as f:
        for idx, line in enumerate(f):
            cols = line.strip().split()
            # Skip lines with exactly 9 columns.
            if len(cols) == 9:
                continue
            try:
                values = [float(x) for x in cols]
                hr_list.append(values)
                valid_indices.append(idx)
            except Exception as e:
                print(f"Skipping invalid HR line {idx+1} in {file_path}: {e}")
    hr = np.array(hr_list)
    
    if apply_noise:
        hr = hr + np.random.normal(loc=0.0, scale=0.16, size=hr.shape)
    
    if scaler is None:
        min_val = hr.min()
        max_val = hr.max()
        if max_val - min_val == 0:
            hr_scaled = hr
        else:
            hr_scaled = (hr - min_val) / (max_val - min_val)
        scaler_params = {'minmax': (min_val, max_val)}
    else:
        min_val, max_val = scaler['minmax']
        if max_val - min_val == 0:
            hr_scaled = hr
        else:
            hr_scaled = (hr - min_val) / (max_val - min_val)
        scaler_params = scaler
    
    return hr_scaled, scaler_params, valid_indices


In [11]:
def load_full_dataset(dataset_path,type_, scalers=None):
    """
    Load and preprocess the full dataset.

    For each feature (shape, el, dist, hr):
      - Reads the file.
      - For training: adds Gaussian noise (σ=0.4), computes min–max scaling.
      - For test: uses the learned min–max scaling parameters from training.
    Computes the common valid indices and combines the features.
    
    Returns:
        data_X: Combined 4D array (samples, features, timesteps, channels).
        classification: Filtered classification labels.
        scalers: Dictionary of scaling parameters for each feature (for training) or None (for test).
    """
    # Load classification labels.
    class_file = os.path.join(dataset_path, type_, "classification.txt")
    classification = np.loadtxt(class_file).reshape(-1, 1)
    classification_indices = list(range(len(classification)))
    
    # Use noise only for training.
    apply_noise_flag = True if type_ == 'train' else False
    
    # Process HR using its special handler.
    if scalers is None or 'hr' not in scalers:
        hr_scaled, hr_scaler, hr_valid = read_and_minmax_hr(dataset_path, type_, apply_noise=apply_noise_flag, scaler=None)
    else:
        hr_scaled, hr_scaler, hr_valid = read_and_minmax_hr(dataset_path, type_, apply_noise=False, scaler=scalers['hr'])
    
    # Process other features: shape, el, dist.
    feat_names = ['shape', 'el', 'dist']
    feature_data = {}
    feature_scalers = {}
    feature_valid = {}
    for feat in feat_names:
        if scalers is None or feat not in scalers:
            data, scaler_param, valid_idx = read_and_minmax_file(dataset_path, type_, feat, apply_noise=apply_noise_flag, scaler=None)
        else:
            data, scaler_param, valid_idx = read_and_minmax_file(dataset_path, type_, feat, apply_noise=False, scaler=scalers[feat])
        feature_data[feat] = data
        feature_scalers[feat] = scaler_param
        feature_valid[feat] = valid_idx
    
    # Determine common valid indices among all features and classification.
    common_valid = set(classification_indices)
    common_valid = common_valid.intersection(set(hr_valid))
    for feat in feat_names:
        common_valid = common_valid.intersection(set(feature_valid[feat]))
    common_valid = sorted(common_valid)
    if len(common_valid) == 0:
        raise ValueError("No common valid indices across all features!")
    
    # Filter each array using the common valid indices.
    classification = classification[common_valid]
    hr_scaled = hr_scaled[common_valid]
    for feat in feat_names:
        feature_data[feat] = feature_data[feat][common_valid]

    # Validate shapes: all features must match in sample count and timesteps.
    def validate_feature_shapes(features_dict):
        first_shape = None
        #print(type_)
        for name, arr in features_dict.items():
            #print(name,arr.shape)
            if first_shape is None:
                first_shape = (arr.shape[0], arr.shape[1])
            else:
                if (arr.shape[0], arr.shape[1]) != first_shape:
                    raise ValueError(f"Shape mismatch in {name}: Expected {first_shape}, got {(arr.shape[0], arr.shape[1])}")
    validate_feature_shapes({feat: feature_data[feat] for feat in feat_names})
    validate_feature_shapes({'hr': hr_scaled, **{feat: feature_data[feat] for feat in feat_names}})
    
    # Combine features along a new axis.
    # New order: (shape, dist, el, hr)
    data_X = np.stack([feature_data['shape'], feature_data['dist'], feature_data['el'], hr_scaled], axis=1)
    # Reshape for CNN input: (samples, features, timesteps, channels)
    data_X = data_X.reshape(data_X.shape[0], data_X.shape[1], data_X.shape[2], 1)
    
    # For training data, return the computed scalers; for test, return None.
    if scalers is None:
        scalers_out = {
            'hr': hr_scaler,
            'shape': feature_scalers['shape'],
            'el': feature_scalers['el'],
            'dist': feature_scalers['dist']
        }
    else:
        scalers_out = None
    return data_X, classification, scalers_out


In [12]:
#########################################
# CNN Model Definition
#########################################
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=9, kernel_size=(4,1), input_shape=input_shape,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=3, kernel_size=(1,3), activation='relu')) #(1,5)

    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:

def find_optimal_threshold(model, X_val, y_val):
    """Find optimal threshold using ROC curve analysis"""
    y_val_pred = model.predict(X_val)
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_val, y_val_pred)
    # Calculate Youden's J statistic
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    optimal_threshold = thresholds[optimal_idx]
    # Calculate AUC
    roc_auc = roc_auc_score(y_val, y_val_pred)
    return optimal_threshold, fpr, tpr, roc_auc

def calculate_metrics(y_true, y_pred, threshold=0.5):
    """Calculate all metrics for a given threshold"""
    y_pred_binary = (y_pred > threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()
    metrics = {
        'confusion_matrix': cm,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'precision': tp/(tp+fp) if (tp+fp) > 0 else 0,
        'recall': tp/(tp+fn) if (tp+fn) > 0 else 0,
        'f1': 2*tp/(2*tp + fp + fn) if (2*tp + fp + fn) > 0 else 0,
        'accuracy': (tp + tn)/(tp + tn + fp + fn),
        'threshold': threshold,
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    return metrics


In [14]:
def plot_roc_curve(fpr, tpr, roc_auc, threshold,file_name="test"):
    """Plot ROC curve with optimal threshold marker"""
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.scatter(fpr[np.argmax(tpr - fpr)], tpr[np.argmax(tpr - fpr)], 
                color='red', label=f'Optimal Threshold ({threshold:.3f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig(f'../static/roc_curves/phase_1/{file_name}.png')
    plt.close()


In [15]:

def run_experiment():
    save_model = 1
    model_path = '../models/1st_phase'

    result_file_path = '../static/phase_1_results_32.csv'
    directory_ = "../Archive/generated_data_achive/training_folders"
    #directory = "../dataset"
    # start the for loop for filename and string concat for the complete path
    #directory_ = "../generated_data/training_folders"
    for file_path in tqdm(os.listdir(directory_)):
        dataset_path = directory_ + "/" +file_path
    

        try:
            # Load and split data
            trainX, trainy, scalers = load_full_dataset(dataset_path,'train')
            #print(trainX.shape,trainy.shape)
            # Split training data into train/validation
            split_idx = int(len(trainX) * 0.8)
            X_train, X_val = trainX[:split_idx], trainX[split_idx:]
            y_train, y_val = trainy[:split_idx], trainy[split_idx:]
            
            # Build and train model
            model = build_model(trainX.shape[1:])
            es = EarlyStopping(monitor='val_accuracy', patience=200, restore_best_weights=True)
            
            history = model.fit(
                X_train, y_train,
                epochs=300,
                batch_size=32,
                validation_data=(X_val, y_val),
                callbacks=[es],
                verbose=0
            )

            if save_model:
                model.save('{}/{}.keras'.format(model_path, file_path))
            
            # Find optimal threshold using ROC analysis
            optimal_threshold, fpr, tpr, roc_auc = find_optimal_threshold(model, X_val, y_val)
            plot_roc_curve(fpr, tpr, roc_auc, optimal_threshold,file_path)
            
            # Load and evaluate test data
            testX, testy, _ = load_full_dataset(dataset_path,'test', scalers)
        
            y_test_pred = model.predict(testX)
            
            # Calculate metrics with optimal threshold
            test_metrics = calculate_metrics(testy, y_test_pred, optimal_threshold)
            # Print comprehensive results
            # make test_metrics["confusion_matrix"] as string
            test_metrics["File"] = file_path
            test_metrics["confusion_matrix"] = str(test_metrics["confusion_matrix"]) 
            test_metrics = {key: [value] for key, value in test_metrics.items()}

            result_df = pd.DataFrame(test_metrics)

            #print(result_df.info())
            # Check if the file exists
            if not os.path.exists(result_file_path):
                # If it doesn't exist, save the current DataFrame
                result_df.to_csv(result_file_path, index=False)
            else:
                # If it exists, read the existing file
                existing_df = pd.read_csv(result_file_path)
                
                # Append the current DataFrame to the existing one
                combined_df = pd.concat([existing_df, result_df], ignore_index=True)
                
                # Save the combined DataFrame back to the file
                combined_df.to_csv(result_file_path, index=False)
        
            
        except Exception as e:
            print(f"\nError: {str(e)}")
            raise

if __name__ == "__main__":
    run_experiment()

  0%|          | 0/28 [00:00<?, ?it/s]

/Users/pulkit/Desktop/mini_project/miniproject/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


/Users/pulkit/Desktop/mini_project/miniproject/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step


...

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step


/Users/pulkit/Desktop/mini_project/miniproject/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step


/Users/pulkit/Desktop/mini_project/miniproject/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step


In [16]:
df = pd.read_csv('../static/phase_1_results_32.csv')
# Find the row with the highest accuracy
top_accuracy_row = df.loc[df['accuracy'].idxmax()]

top_accuracy_row

confusion_matrix    [[505 147]\n [135 495]]
tp                                      495
tn                                      505
fp                                      147
fn                                      135
precision                          0.771028
recall                             0.785714
f1                                 0.778302
accuracy                           0.780031
threshold                          0.503254
roc_auc                            0.851195
File                 L_hydrophobicity_scale
Name: 51, dtype: object

In [2]:
import pandas as pd
df = pd.read_csv('../static/phase_2_results_32.csv')
# Find the row with the highest accuracy
top_accuracy_row = df.loc[df['accuracy'].idxmax()]

top_accuracy_row

confusion_matrix    [[514 138]\n [149 481]]
tp                                      481
tn                                      514
fp                                      138
fn                                      149
precision                           0.77706
recall                             0.763492
f1                                 0.770216
accuracy                           0.776131
threshold                          0.506026
roc_auc                            0.853717
File                             KUHL950101
Name: 16, dtype: object